# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [113]:
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import SubElement
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( './data/mondial_database.xml' )

In [31]:
s = pd.Series()
for country in document.findall('country'):
    infantmortality= country.find('infant_mortality')
    name = country.find('name')
    s.set_value(name.text, infantmortality.text)
res = pd.to_numeric(s, errors='coerce')
res.sort_values().head(10)

Monaco            1.81
Japan             2.13
Bermuda           2.48
Norway            2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
dtype: float64

In [93]:
df = pd.DataFrame(columns=['Year', 'City', 'Population'])
for city in document.findall('country/province/city'):
    cityname = city.find('name').text
    for populationinfo in city.findall('population'):
        year = populationinfo.attrib['year']
        population = populationinfo.text
        idx = len(df)
        df.loc[idx] = [year, cityname, population]
df[['Year', 'Population']] = df[['Year', 'Population']].apply(pd.to_numeric)
df.groupby(['City'], sort=False)['Year', 'Population'].max().sort_values(by=['Population'], ascending=False).head(10)

,year,population
city,,
Shanghai,2010,22315474
Istanbul,2012,13710512
Delhi,2011,12877470
Mumbai,2011,12442373
Moskva,2013,11979529
Beijing,2010,11716620
São Paulo,2010,11152344
Tianjin,2010,11090314
Guangzhou,2010,11071424


In [101]:
df = pd.DataFrame(columns=['Year', 'Ethnic Group','Population'])
i=0
for country in document.findall('country'):
    for populationinfo in country.findall('population'):
        for ethnicgroup in country.findall('ethnicgroup'):
            ethnicgrouppercentage = (float)(ethnicgroup.attrib['percentage'])
            year = populationinfo.attrib['year']
            population = (int(populationinfo.text))*(ethnicgrouppercentage/100)
            df.loc[i] = [year, ethnicgroup.text, population]
            i = i+1
df.groupby(['Ethnic Group'], sort=False)['Year', 'Population'].max().sort_values(by=['Population'], ascending=False).head(10)

,Year,Population
Ethnic Group,,
Han Chinese,2013,1.245059e+09
Indo-Aryan,2011,8.718156e+08
Dravidian,2011,3.027137e+08
European,2014,2.549581e+08
African,2014,1.626516e+08
Bengali,2011,1.467769e+08
Japanese,2013,1.272890e+08
Russian,2014,1.182464e+08
Javanese,2014,1.134560e+08


In [131]:
df_countries = pd.DataFrame(columns=['Country', 'CountryCode'])
i=0
for country in document.findall('country'):
    countryname =country.find('name').text
    shortcode = country.attrib['car_code']
    df_countries.loc[i] = [countryname, shortcode]
    i=i+1

In [140]:
df_rivers = pd.DataFrame(columns=['River', 'Length','CountryCode'])
i=0
for river in document.findall('river'):
    rivername = river.find('name').text
    riverlength = river.find('length')
    if riverlength is None:
        continue
    rivercountry = river.attrib['country']
    df_rivers.loc[i] = [rivername, riverlength.text, rivercountry]
    i=i+1
join = pd.merge(df_rivers, df_countries, on=['CountryCode'])[['River', 'Length', 'Country']]
join[['Length']] = join[['Length']].apply(pd.to_numeric)
join.sort_values(by=['Length'], ascending=False).head(10)

,River,Length,Country
75,Jangtse,6380.0,China
74,Hwangho,4845.0,China
69,Lena,4400.0,Russia
87,Missouri,4130.0,United States
66,Jenissej,4092.0,Russia
86,Mississippi,3778.0,United States
64,Ob,3650.0,Russia
61,Volga,3531.0,Russia
73,Tarim-Yarkend,3260.0,China
98,Rio Sao Francisco,2830.0,Brazil


In [141]:
df_lakes = pd.DataFrame(columns=['Lake', 'Area','CountryCode'])
i=0
for lake in document.findall('lake'):
    lakename = lake.find('name').text
    lakearea = lake.find('area')
    if lakearea is None:
        continue
    lakecountry = lake.attrib['country']
    df_lakes.loc[i] = [lakename, lakearea.text, lakecountry]
    i=i+1
join = pd.merge(df_lakes, df_countries, on=['CountryCode'])[['Lake', 'Area', 'Country']]
join[['Area']] = join[['Area']].apply(pd.to_numeric)
join.sort_values(by=['Area'], ascending=False).head(10)

,Lake,Area,Country
75,Lake Michigan,57800.0,United States
68,Great Bear Lake,31792.0,Canada
35,Ozero Baikal,31492.0,Russia
69,Great Slave Lake,28568.0,Canada
71,Lake Winnipeg,24420.0,Canada
43,Ozero Balchash,18428.0,Kazakhstan
33,Ozero Ladoga,18400.0,Russia
93,Lake Maracaibo,13000.0,Venezuela
66,Makarikari Salt Pan,12000.0,Botswana
96,Salar de Uyuni,10582.0,Bolivia


In [151]:
df1 = pd.DataFrame(columns=['City', 'CityCode', 'CountryCode'])
i=0
for city in document.findall('country/city'):
    cityname = city.find('name').text
    countrycode = city.attrib['country']
    citycode = city.attrib['id']
    df1.loc[i] =[cityname, citycode, countrycode]
    i=i+1
df_cities = pd.merge(df1, df_countries, on=['CountryCode'])[['City', 'CityCode', 'Country']]
df_cities

,City,CityCode,Country
0,Tirana,cty-Albania-Tirane,Albania
1,Shkodër,stadt-Shkoder-AL-AL,Albania
2,Durrës,stadt-Durres-AL-AL,Albania
3,Vlorë,stadt-Vlore-AL-AL,Albania
4,Elbasan,stadt-Elbasan-AL-AL,Albania
5,Korçë,stadt-Korce-AL-AL,Albania
6,Skopje,cty-Macedonia-Skopje,Macedonia
7,Kumanovo,cty-Macedonia-Kumanovo,Macedonia
8,Beograd,city-Belgrade-SRB-SRB,Serbia
9,Novi Sad,city-NoviSad-SRB-SRB,Serbia


In [165]:
df_airports = pd.DataFrame(columns=['Airport', 'AirportCode', 'AirportCity', 'Elevation','CountryCode'])
i=0
for airport in document.findall('airport'):
    airportname = airport.find('name').text
    airportelevation = airport.find('elevation')
    if airportelevation is None:
        continue
    airportcountry = airport.attrib['country']
    airportcode = airport.attrib['iatacode']
    if 'city' not in airport.attrib:
        airportcity = np.NaN
    else:
        airportcity = airport.attrib['city']
    df_airports.loc[i] = [airportname, airportcode, airportcity, airportelevation.text, airportcountry]
    i=i+1
join1 = pd.merge(df_airports, df_countries, on=['CountryCode'])[['Airport', 'AirportCode', 'AirportCity', 'Elevation', 'Country']]
join1[['Elevation']] = join[['Elevation']].apply(pd.to_numeric)
join1.sort_values(by=['Elevation'], ascending=False).head(10)

join2 = pd.merge(join1, df_cities, left_on=['AirportCity', 'Country'], right_on=['CityCode', 'Country'], how='left')[['Airport', 'AirportCode', 'City', 'Elevation', 'Country']]
join2.sort_values(by='Elevation', ascending=False).head(10)

,Airport,AirportCode,City,Elevation,Country
80,El Alto Intl,LPB,NaN,4063.0,Bolivia
219,Lhasa-Gonggar,LXA,NaN,4005.0,China
241,Yushu Batang,YUS,NaN,3963.0,China
813,Juliaca,JUL,NaN,3827.0,Peru
815,Teniente Alejandro Velasco Astete Intl,CUZ,NaN,3311.0,Peru
82,Juana Azurduy De Padilla,SRE,NaN,2905.0,Bolivia
334,Mariscal Sucre Intl,UIO,Quito,2813.0,Ecuador
805,Coronel Fap Alfredo Mendivil Duarte,AYP,NaN,2719.0,Peru
807,Mayor General FAP Armando Revoredo Iglesias Ai...,CJA,NaN,2677.0,Peru
692,Licenciado Adolfo Lopez Mateos Intl,TLC,NaN,2581.0,Mexico
